### setup

In [1]:
import os
import argparse
import pickle
import time

import numpy as np; np.seterr(invalid='ignore')
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import TensorDataset, DataLoader

In [2]:
parser = {
    'data_path': '../data/wttsf/',
    'train_file': 'train_1.csv',
    'intermediate_path': '../intermediate/',
    'n_epoch': 3,
    'future': 73,
    'batch_size': 64,
    'hidden_size': 256,
    'log_every': 10,
    'read_from_file': False,
    'train': True,
    'model_name': '',
    'cuda': True,
    'seed': 20170903,
}
args = argparse.Namespace(**parser)

args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)

args.intermediate_path = os.path.join(args.intermediate_path, str(args.seed))

### model

In [3]:
class DenseLSTMForecast(nn.Module):
    def __init__(self, hidden_size):
        super(DenseLSTMForecast, self).__init__()
        self.lstm1 = nn.LSTMCell(15, hidden_size)
        self.lstm2 = nn.LSTMCell(hidden_size+15, hidden_size)
        self.lstm3 = nn.LSTMCell(2*hidden_size+15, hidden_size)
        self.linear = nn.Linear(3*hidden_size+15, 1)
        self.hidden_size = hidden_size

    def forward(self, x, feature, future=1):
        o = []
        tt = torch.cuda if args.cuda else torch
        h1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        
        for x_t in x.chunk(x.size(1), dim=1):
            x_t = x_t.squeeze(dim=1)
            xd_t = torch.cat([x_t, feature], dim=1)
            h1_t, c1_t = self.lstm1(xd_t, (h1_t, c1_t))
            h1d_t = torch.cat([xd_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([xd_t, h1_t, h2_t], dim=1)
            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
            h3d_t = torch.cat([xd_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h3d_t)
            o.append(o_t)
            
        for i in range(future-1):
            od_t = torch.cat([o_t, feature], dim=1)
            h1_t, c1_t = self.lstm1(od_t, (h1_t, c1_t))
            h1d_t = torch.cat([od_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([od_t, h1_t, h2_t], dim=1)
            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
            h3d_t = torch.cat([od_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h3d_t)
            o.append(o_t)

        return torch.stack(o, dim=1)

### utils

In [4]:
def smape(y_pred, y_true):
    y_pred = np.around(np.clip(np.exp(y_pred)-1, 0, None))
    y_true = np.around(np.exp(y_true) - 1)
    raw_smape = np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))
    kaggle_smape = np.nan_to_num(raw_smape)
    return np.mean(kaggle_smape) * 200

In [5]:
def get_data():
    scaled_data_file = os.path.join(args.intermediate_path,
                                    'scaled_data.pkl')
    scaler_file = os.path.join(args.intermediate_path, 'scaler.pkl')
    features_file = os.path.join(args.intermediate_path, 'features.pkl')
    
    if not args.read_from_file:
        data_df = pd.read_csv(os.path.join(args.data_path, args.train_file),
                              index_col='Page')
        data_df["agent"] = data_df.index.str.rsplit('_').str.get(-1)
        data_df["access"] = data_df.index.str.rsplit('_').str.get(-2)
        data_df["project"] = data_df.index.str.rsplit('_').str.get(-3)
        features = pd.get_dummies(data_df[["agent", "access", "project"]],
            columns=["agent", "access", "project"]).values.astype('float32')
        raw_data = np.nan_to_num(
            data_df.iloc[:,:-3].values.astype('float32'))
        data = np.log1p(raw_data)
        scaler = StandardScaler()
        scaler.fit(np.swapaxes(data[:, :-args.future], 0, 1))
        scaled_data = scaler.transform(np.swapaxes(data, 0, 1))
        scaled_data = np.swapaxes(scaled_data, 0, 1)
        
        with open(scaled_data_file, 'wb') as f:
            pickle.dump(scaled_data, f)
        with open(scaler_file, 'wb') as f:
            pickle.dump(scaler, f)
        with open(features_file, 'wb') as f:
            pickle.dump(features, f)
    else:
        with open(scaled_data_file, 'rb') as f:
            scaled_data = pickle.load(f)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
        with open(features_file, 'rb') as f:
            features = pickle.load(f)
    return scaled_data, scaler, features

In [6]:
def train(scaled_data, scaler, features, model, criterion, optimizer):
    p = np.random.permutation(scaled_data.shape[0])
    inverse_p = np.argsort(p)

    input_tensor = torch.from_numpy(scaled_data[p, :-1]).unsqueeze(2)
    target_tensor = torch.from_numpy(scaled_data[p, 1:]).unsqueeze(2)
    features_tensor = torch.from_numpy(features[p, :])
    dataset = TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    train_loss = 0
    val_output_list = []
    init_time = time.time()
    for i, (inputt, target) in enumerate(data_loader):
        feature = features_tensor[i*args.batch_size:(i*args.batch_size
                                                     +inputt.size(0))]
        if args.cuda:
            inputt = inputt.cuda()
            target = target.cuda()
            feature = feature.cuda()
        inputt = Variable(inputt)
        target = Variable(target)
        feature = Variable(feature)
        
        output = model(inputt, feature)
        pos = np.random.randint(args.future,
                                output.size(1)-2*args.future+1)
        loss = criterion(output[:, pos:pos+args.future],
                         target[:, pos:pos+args.future])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0] * inputt.size(0)
        val_output_list.append(output[:, -args.future:]
                               .data.squeeze(2).cpu().numpy())

        if i % args.log_every == 0:
            print("   % Time: {:4.0f}s | Batch: {:4} | "
                  "Train loss: {:.4f}".format(
                      time.time()-init_time, i+1, loss.data[0]))
        
    val_output_all = np.concatenate(val_output_list, axis=0)[inverse_p]
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(val_output_all, 0, 1)), 0, 1)
    
    var_target = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(scaled_data[:, -args.future:], 0, 1)), 0, 1)
    train_loss /= scaled_data.shape[0]
    val_loss = smape(prediction, var_target)
    
    print("="*10)
    print("   % Epoch: {} | Time: {:4.0f}s | "
          "Train loss: {:.4f} | Val loss: {:.4f}"
          .format(epoch, time.time()-init_time, train_loss , val_loss))
    print("="*10)
    return val_loss

In [7]:
def forecast(scaled_data, scaler, features, model):
    input_tensor = torch.from_numpy(scaled_data).unsqueeze(2)
    target_tensor = torch.zeros(input_tensor.size(0))
    features_tensor = torch.from_numpy(features)
    dataset = torch.utils.data.TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    output_list = []
    for i, (inputt, _) in enumerate(data_loader):
        feature = features_tensor[i*args.batch_size:(i*args.batch_size
                                                     +inputt.size(0))]
        if args.cuda:
            inputt = inputt.cuda()
            feature = feature.cuda()
        inputt = Variable(inputt)
        feature = Variable(feature)
        output = model(inputt, feature, args.future)
        output_list.append(output.data.squeeze(2).cpu().numpy()
                           [:, -args.future:])
        
    output_all = np.concatenate(output_list, axis=0)
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(output_all, 0, 1)), 0, 1)
    prediction = np.around(np.clip(np.exp(prediction) - 1, 0, None))
    return prediction

In [8]:
def save_model(model, epoch, loss):
    model_file = os.path.join(args.intermediate_path,
                              "model_seed{}_epoch{}_loss_{:.4f}.pth"
                              .format(args.seed, epoch, loss))
    torch.save(model.state_dict(), os.path.join(model_file))

### prepare

In [9]:
scaled_data, scaler, features = get_data()

In [10]:
model = DenseLSTMForecast(args.hidden_size)
if args.cuda:
    model.cuda()
criterion = nn.L1Loss()

In [ ]:
optimizer = optim.Adam(model.parameters())
scheduler = MultiStepLR(optimizer, milestones=[2])

### train

In [ ]:
if args.train:
    for epoch in range(1, args.n_epoch+1):
        print("=> EPOCH {}".format(epoch))
        scheduler.step()
        val_loss = train(scaled_data, scaler, features,
                         model, criterion, optimizer)
else:
    model_file = os.path.join(args.intermediate_path, args.model_name)
    model.load_state_dict(torch.load(model_file))

=> EPOCH 1
   % Time:    1s | Batch:    1 | Train loss: 0.6661
   % Time:    9s | Batch:   11 | Train loss: 0.4750
   % Time:   17s | Batch:   21 | Train loss: 0.4477
   % Time:   25s | Batch:   31 | Train loss: 0.4532
   % Time:   32s | Batch:   41 | Train loss: 0.4209
   % Time:   40s | Batch:   51 | Train loss: 0.4335
   % Time:   48s | Batch:   61 | Train loss: 0.4341
   % Time:   56s | Batch:   71 | Train loss: 0.4560
   % Time:   64s | Batch:   81 | Train loss: 0.4376
   % Time:   71s | Batch:   91 | Train loss: 0.4215
   % Time:   79s | Batch:  101 | Train loss: 0.4257
   % Time:   87s | Batch:  111 | Train loss: 0.4364
   % Time:   95s | Batch:  121 | Train loss: 0.4359
   % Time:  103s | Batch:  131 | Train loss: 0.3663
   % Time:  110s | Batch:  141 | Train loss: 0.4097
   % Time:  118s | Batch:  151 | Train loss: 0.3885
   % Time:  126s | Batch:  161 | Train loss: 0.4422
   % Time:  134s | Batch:  171 | Train loss: 0.4931
   % Time:  142s | Batch:  181 | Train loss: 0.4117
 

   % Time: 1231s | Batch: 1581 | Train loss: 0.3892
   % Time: 1238s | Batch: 1591 | Train loss: 0.3885
   % Time: 1246s | Batch: 1601 | Train loss: 0.4246
   % Time: 1254s | Batch: 1611 | Train loss: 0.3516
   % Time: 1262s | Batch: 1621 | Train loss: 0.3586
   % Time: 1270s | Batch: 1631 | Train loss: 0.3643
   % Time: 1278s | Batch: 1641 | Train loss: 0.3496
   % Time: 1286s | Batch: 1651 | Train loss: 0.3979
   % Time: 1293s | Batch: 1661 | Train loss: 0.3914
   % Time: 1301s | Batch: 1671 | Train loss: 0.3768
   % Time: 1309s | Batch: 1681 | Train loss: 0.3821
   % Time: 1317s | Batch: 1691 | Train loss: 0.4067
   % Time: 1325s | Batch: 1701 | Train loss: 0.4200
   % Time: 1333s | Batch: 1711 | Train loss: 0.3575
   % Time: 1341s | Batch: 1721 | Train loss: 0.3861
   % Time: 1349s | Batch: 1731 | Train loss: 0.3698
   % Time: 1356s | Batch: 1741 | Train loss: 0.4216
   % Time: 1364s | Batch: 1751 | Train loss: 0.4430
   % Time: 1372s | Batch: 1761 | Train loss: 0.3778
   % Time: 1

### test

In [ ]:
prediction = forecast(scaled_data, scaler, features, model)
prediction_file = os.path.join(args.intermediate_path,
                               'prediction_seed{}.pkl'.format(args.seed))

with open(prediction_file, 'wb') as f:
    pickle.dump(prediction, f)